In [ ]:
import os
import re
from datetime import datetime
from ltlcross_runner import LtlcrossRunner
from tools_hier import get_tools, ltl3dra_tools, full_tools, tool_order
from evaluation_utils import sort_by_tools, split_cols
import pandas as pd
import spot
spot.setup()

# Comparison of LTL to deterministic automata translators 

$\newcommand{\F}{\mathsf{F}}\newcommand{\G}{\mathsf{G}}\newcommand{\U}{\mathsf{U}}\newcommand{\X}{\mathsf{X}}$
## Tools
We have 
* tools that translate \[fragments\] of LTL into (generalized) Rabin automata:
 - [Rabinizer 3.1](https://www7.in.tum.de/~kretinsk/rabinizer3.html)
 - [Rabinizer 4](https://www7.in.tum.de/~kretinsk/rabinizer4.html)
 - [LTL3DRA](https://sourceforge.net/projects/ltl3dra/) v.0.2.6 \[LTL$\smallsetminus\G$($\U\X$)\]
* tools that chain translation of LTL into cut-deterministic Büchi automata (aka LDBA) or into deterministic Rabin automata with a construction that create deterministic parity automaton. They avoid Safra-like determinization. Both are from the [owl/Rabinizer4 library](https://www7.in.tum.de/~kretinsk/rabinizer4.html)
 - ltl2dpa --mode=ldba
 - ltl2dpa --mode=rabinizer (relies on transition-based index appearance record)
* tools that perform Safra-based determinization of N\[G\]BA into {Rabin/Parity} automata that use intermediete NBA
 - [ltl2dstar](http://ltl2dstar.de) \[NBA only\] {Rabin)}
 - autfilt from [Spot](https://spot.lrde.epita.fr/) 2.5 {parity}
 - here we use the following LTL2TGBA/LTL2NBA translators:
   - ltl2tgba from [Spot](https://spot.lrde.epita.fr/) 2.5
   - [LTL3BA](https://sourceforge.net/p/ltl3ba/) v. 1.1.3; we use LTL3BA in two settings:
     - `ltl3ba` {LTL3BA}
     - `ltl3ba -d` {LTL3BAd} which prefer more deterministic automata to smaller ones
 - we further use experimentaly for autfilt the LTL to Emerson-Lei automata translator
   - [LTL3TELA](https://github.com/jurajmajor/ltl3tela) v.1.1.1
* Safra's based determinization into deterministic parity automata that uses information about the LTL formula
 - `ltl2tgba -DG` from [Spot](https://spot.lrde.epita.fr/) 2.5

In [ ]:
runners = {}
cols=["states","transitions","acc","time","nondet_states"]
for source in ('literature','random'):
    for t in ('full','ltl3dra'):
        name = '{}_{}'.format(source,t)
        tools = full_tools if t=='full' else ltl3dra_tools
        runners[name] = \
            LtlcrossRunner(tools,\
                    res_filename='data/{}.csv'.format(name),\
                    formula_files=['formulae/{}.ltl'.format(name)],\
                    cols=cols)

name = 'random_fg'
tools = ltl3dra_tools
runners[name] = LtlcrossRunner(tools,\
                    res_filename='data/{}.csv'.format(name),\
                    formula_files=['formulae/{}.ltl'.format(name)],\
                    cols=cols)

In [ ]:
for name, r in runners.items():
    print('{}: Working on {}'.format(datetime.now().strftime('[%d.%m.%Y %T]'),name))
    r.name = name
    r.parse_results()
    r.na_incorrect()
    r.orig_count = len(r.values)
    r.clean_count = len(r.values.dropna())

In [ ]:
for name, r in runners.items():
    print('The benchmark {} started with {} formulae and has valid data for {} formulae'.format(name,r.orig_count,r.clean_count))
    r.errors = {}
    for e in ['timeout', 'parse error', 'incorrect', 'crash', 'no output']:
        s = r.get_error_count(e).sum()
        if not pd.isna(s):
            r.errors[e] = r.get_error_count(e).sum()
    s = pd.Series(r.errors, name='count')
    s.index.name = 'error type'
    display(s.reset_index())

## Gather data

In [ ]:
def compute_runner(self,cols=['states','acc','time'],
                   tool_subset=None, split_col_names=True, col_lev_names=None,
                   counts = True):
    '''Returns a dataframe with rows indexed by tools in
    `tool_subset (or `self.tools.keys()`) and columns by
    cols. The values are cummulative sums for respective
    tool and column.'''
    if tool_subset is None:
        tool_subset = self.tools.keys()
    name = self.name
    name = name.replace('ltl3dra','ltl-gux')
    if counts:
        name = '{} ({})'.format(name,self.clean_count)
    col_names = ['{}_{}'.format(name,c) for c in cols]
    df = pd.DataFrame(columns=col_names,index=tool_subset)
    for col in cols:
        df['{}_{}'.format(name,col)] = self.cummulative(col).map(lambda x: '%0.0f' % x)
    if split_col_names:
        res = split_cols(df,axis=1,symbol='_',names=col_lev_names)
    else:
        res = df
    return res 

In [ ]:
display(compute_runner(r).head())
display(compute_runner(r,split_col_names=False).head())
compute_runner(r,col_lev_names=['source','fragment','metric']).head()

In [ ]:
data = []
order = [
    ('literature','full'),
    ('literature','ltl3dra'),
    ('random','full'),
    ('random','ltl3dra'),
    ('random','fg'),
]
for o in order:
    data.append(compute_runner(runners['{}_{}'.format(o[0],o[1])]))
res = pd.concat(data,axis=1)
res = split_cols(res,axis=0,symbol='/')

In [ ]:
def fix_acc(acc):
    acc = acc.replace('TGBA','TGB').replace('NBA','SB')
    acc = acc.replace('DPA','DTPA').replace('DTPA','TP')
    acc = acc.replace('DSRA','SR').replace('DTGRA','TGR')
    return acc.replace('DTELA','TEL').replace('TEL.TP','TEL.TEL')

In [ ]:
def fix_tools(tool):
    tool = tool.replace('ltl2dstar(NBA)',
                        '\parbox[c]{1.3cm}{\centering ltl2dstar NBA}')
    tool = tool.replace('Spot',
                        '\rotatebox[origin=c]{90}{Spot (autfilt)}')
    tool = tool.replace('R3','Rabinizer 3').replace('R4','Rabinizer 4')
    return tool

In [ ]:
def fix_fragment(fr):
    fr = fr.replace('full','full LTL')
    fr = fr.replace('ltl-gux','\LTLGUX')
    return fr.replace('fg','\FG').replace('(','[$').replace(')','$]')

In [ ]:
def shorten_index(df):
    i = [(fix_tools(t[0]),t[1],fix_acc(t[2])) for t in df.index.values]
    df.index=pd.MultiIndex.from_tuples(i)
    ci = [(t[0],fix_fragment(t[1]),t[2]) for t in df.columns.values]
    df.columns=pd.MultiIndex.from_tuples(ci)
    

In [ ]:
def add_type_lines(filename):
    cline = '\midrule'
    todo = ['\multirow{2}{*}{ltl2dpa}','\multirow{3}{*}{ltl2dstar}']
    for rplc in todo:
        with open(filename) as f:
            lines = f.read().replace("{}".format(rplc),
                                     "{}{}".format(cline,rplc))
        with open(filename,"w") as f1:
            f1.write(lines)

In [ ]:
def fix_lines(filename, end=12):
    end = str(end)
    todo = [('\multirow{2}{*}{ltl2dpa}','\cmidrule{1-'+ end +'}'),
     ('& \multirow{2}{*}{Spot}','\cmidrule{2-'+ end +'}')]
    for rplc, cline in todo:
        with open(filename) as f:
            lines = f.read().replace("{}".format(rplc),
                                     "{}{}".format(cline,rplc))
        with open(filename,"w") as f1:
            f1.write(lines)

In [ ]:
def color_table(filename):
    setup = '''\\newcolumntype{a}{>{\\columncolor{blue!20}}r}
\\newcolumntype{b}{>{\\columncolor{blue!20}}c}
\\setlength{\\aboverulesep}{0pt}
\\setlength{\\belowrulesep}{0pt}
\\setlength{\\extrarowheight}{.75ex}
\\setlength{\\heavyrulewidth}{2pt}
\\setlength{\\lightrulewidth}{1.2pt}
\\def\\high{\\cellcolor{darkgreen!40}}
'''
    with open(filename) as f:
        lines = f.read()
    lines = fix_header_colors(lines)
    lines = lines.replace('cline','cmidrule')
    lines = lines.replace('$nan$','---')
    # Remove cmidrules in front of midrule
    lines = re.sub(r"(\\cmidrule\{\d+-\d+\}\n?)+\\midrule", "\\midrule", lines)
    with open(filename,"w") as f1:
        f1.write(setup + lines)

In [ ]:
def make_heading(filename):
    with open(filename) as f:
        lines = f.read()
    lines = re.sub(r"(\s+)&\s+&\s+&\s+states", r"\1main tool & intermediate & acc & states", lines)
    with open(filename,"w") as f1:
        f1.write(lines)

In [ ]:
def fix_header_colors(lines):
    lines = lines.replace('{c}','{b}',1)
    i = lines.find('{c}')
    return lines[:i+1] + lines[i+1:].replace('{c}','{b}',1)

In [ ]:
def cummulative_to_latex(res,file='cumm.tex',transpose=False):
    if transpose:
        res = res.T
    col_f = 'ccr'
    for i in range(len(res.columns)//3):
        if i % 2 == 0:
            col_f += 'aaa'
        else:
            col_f += 'rrr'
    res = res
    res.to_latex(buf=open(file,'w'),multirow=True,escape=False,na_rep='---',float_format=lambda x: '$' + '%0.0f' % x + '$',column_format=col_f,multicolumn_format='c')

In [ ]:
def high_min(data):
    return ['\high ${:0.0f}$'.format(m) if 
            m == data.min() else m for m in data]

In [ ]:
sorted_all = sort_by_tools(res,tool_order)
shorten_index(sorted_all)
sorted_all = sorted_all.astype(float).apply(high_min)

In [ ]:
cummulative_to_latex(sorted_all.random, file='cumm_rand.tex', transpose=False)
fix_lines('cumm_rand.tex',12)
add_type_lines('cumm_rand.tex')
color_table('cumm_rand.tex')
make_heading('cumm_rand.tex')

In [ ]:
cummulative_to_latex(sorted_all.literature, file='cumm_lit.tex', transpose=False)
fix_lines('cumm_lit.tex',9)
add_type_lines('cumm_lit.tex')
color_table('cumm_lit.tex')
make_heading('cumm_rand.tex')